In [7]:
date='20130606'
os.chdir('/Users/Armand/Capstone/ipinyou.contest.dataset/training2nd')
# 2. Define Parameters
header_season2_bids=[
    'Bid ID'
    ,'Timestamp'
    ,'iPinYou ID'
    ,'User-Agent'
    ,'IP'
    ,'Region ID'
    ,'City ID'
    ,'Ad Exchange'
    ,'Domain'
    ,'URL'
    ,'Anonymous URL'
    ,'Ad Slot ID'
    ,'Ad Slot Width'
    ,'Ad Slot Height'
    ,'Ad Slot Visibility'
    ,'Ad Slot Format'
    ,'Ad Slot Floor Price'
    ,'Creative ID'
    ,'Bidding Price'
    ,'Advertiser ID'
    ,'User Profile IDs']

header_season2=[
    'Bid ID'
    ,'Timestamp'
    ,'Log Type'
    ,'iPinYou ID'
    ,'User-Agent'
    ,'IP'
    ,'Region ID'
    ,'City ID'
    ,'Ad Exchange'
    ,'Domain'
    ,'URL'
    ,'Anonymous URL'
    ,'Ad Slot ID'
    ,'Ad Slot Width'
    ,'Ad Slot Height'
    ,'Ad Slot Visibility'
    ,'Ad Slot Format'
    ,'Ad Slot Floor Price'
    ,'Creative ID'
    ,'Bidding Price'
    ,'Paying Price'
    ,'Landing Page URL'
    ,'Advertiser ID'
    ,'User Profile IDs']

unzipped_file = bz2.BZ2File('bid.'+date+'.txt.bz2', "r")
bids=pd.read_table(unzipped_file, names=header_season2_bids)
unzipped_file = bz2.BZ2File('imp.'+date+'.txt.bz2', "r")
impressions=pd.read_table(unzipped_file, names=header_season2)
    
unzipped_file = bz2.BZ2File('clk.'+date+'.txt.bz2', "r")
clicks=pd.read_table(unzipped_file, names=header_season2)
    
unzipped_file = bz2.BZ2File('conv.'+date+'.txt.bz2', "r")
conversions=pd.read_table(unzipped_file, names=header_season2)
all_data={'bids': bids, 'impressions':impressions, 'clicks':clicks, 'conversions':conversions}
for key, value in all_data.items(): 
    print('shape of %s is %s'%(key, value.shape))
#ASSUMPTION 1: SOME COLUMNS CAN BE DROPPED BECAUSE THEY HAVE NO PREDICTIVE POWER
cols_to_drop=['iPinYou ID'
            ,'IP'
            ,'Domain'
            ,'URL'
            ,'Anonymous URL'
            ,'Ad Slot ID'
            ,'Creative ID']
for index, value in all_data.items(): 
    value.drop(cols_to_drop, axis=1, inplace =True)
    if index!='bids': 
        value.drop('Landing Page URL', axis=1, inplace=True)
#PRESSION MERGE 
## ASSUMPTION 2.1:COUNT DUPLICATES ON BID ID FOR IMPRESSIONS AND ADD THEM TO LINE
## ASSUMPTION 2.2: ONLY KEEP COLUMNS THAT ACTUALLY DIFFER IN THE IMPRESSION TABLE
impression_count=impressions.groupby(['Bid ID']).size()
impression_to_merge =pd.merge(impressions[['Bid ID'
        , 'Timestamp'
        , 'User-Agent'
        , 'Log Type'
        , 'Paying Price'
        , 'User Profile IDs']].drop_duplicates('Bid ID'), impression_count.rename('n_impressions'), left_on='Bid ID',right_index=True)
df=pd.merge(bids, impression_to_merge, on="Bid ID", suffixes=('_bid', '_imp'), how="left")
#CKS MERGE
## ASSUMPTION 3.1: COUNT DUPLICATES FOR BID ID FOR CLICKS AND ADD THEM  
## ASSUMPTION 3.2: ONLY KEEP COLUMNS THAT ACTUALLY DIFFER IN THE CLICKS TABLE
clicks_count=clicks.groupby(['Bid ID']).size()
clicks_to_merge =pd.merge(clicks[['Bid ID'
                    , 'Timestamp'
                    , 'User-Agent'
                    , 'User Profile IDs'
                    , 'Region ID'
                    ]].drop_duplicates('Bid ID'), clicks_count.rename('n_clicks'), left_on='Bid ID',right_index=True)     
df=pd.merge(df, clicks_to_merge, left_on="Bid ID",right_on="Bid ID", suffixes=('_bids_join_impressions', '_clicks'), how="left")
# ADD COLUMNS (QUITE REDUNDANT WITH n_clicks & n_impressions)
df['has impression']=~(df['Timestamp_imp'].isna())
df['has click']=~(df['Region ID_clicks'].isna())
#CONVERT BOOL TO INT 
df['has impression']=df['has impression'].astype(int)
df['has click']=df['has click'].astype(int)
#REDUCE SIZE   
cols_to_drop = ['Bid ID']
df.drop(cols_to_drop, axis=1, inplace =True)

shape of bids is (9586949, 21)
shape of impressions is (1821350, 24)
shape of clicks is (1289, 24)
shape of conversions is (43, 24)


In [9]:
df.shape

(9586949, 26)

In [13]:
43/9586949

4.485264290025951e-06